In [6]:
from flask import Flask
from flask.ext import restful # Se instala con sudo pip install flask-restful

In [7]:
# Creamos la aplicación
app = Flask("Test")
api = restful.Api(app)

In [8]:
# Ahora añadimos recursos a la aplicación
class HelloWorld(restful.Resource):
    def get(self):
        return {'Hello': 'World'}

#### Protocolo http:

Request
GET /api/v1/helloworld
PUT
POST /api/v1/helloworld
DELETE, etc

Responde:
Status 500 ==> Error
Status 494 ==> Resource not found

In [9]:
api.add_resource(HelloWorld, '/api/v1/helloworld')

In [10]:
# Definimos ports
app.run(port=8080)

### Otra aplicación con datos dinámicos

In [11]:
import pandas as pd

In [12]:
alturas=[1,2,3,4,5,6]
alturas = pd.Series(alturas)
res = alturas.to_json()

In [13]:
type(res) # Es un string pero quiero un objeto

str

In [14]:
import json
res2 = json.loads(res)
res2

{u'0': 1, u'1': 2, u'2': 3, u'3': 4, u'4': 5, u'5': 6}

In [15]:
# o mejor
res2 = alturas.to_dict()
res2

{0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6}

In [25]:
# Creamos otra aplicación
app = Flask("Test")
api = restful.Api(app)
# Ahora añadimos recursos a la aplicación
class HelloWorld(restful.Resource):
    def get(self):
        return {'helloworld': res2}
api.add_resource(HelloWorld, '/api/v1/helloworld')
# Definimos ports
app.run(port=8080)

### Ahora parseando

In [26]:
# Creamos otra aplicación
from flask_restful import reqparse

app = Flask("Test")
api = restful.Api(app)
# Ahora añadimos recursos a la aplicación
class HelloWorld(restful.Resource):
    def get(self):
        # Argument parsing after ?id=
        parser = reqparse.RequestParser()
        parser.add_argument('id',required=True)
        parser.add_argument('name')
        args = parser.parse_args()
        return {'helloworld': args["id"],"name":args['name']}
api.add_resource(HelloWorld, '/api/v1/helloworld')
# Definimos ports
app.run(port=8080)

## Ahora web service con fichero Temperature y sqlite3

In [65]:
import sqlite3 # he creado antes base de datos worldtemp.db (mirar apuntes)

In [73]:
# Creamos conexión
conn = sqlite3.connect("data/worldtemp.db")
# Creamos cursor
cursor = conn.cursor()

In [74]:
rows = cursor.execute("SELECT * FROM worldtemp LIMIT 1")

In [75]:
for row in rows:
    print row

(0, 51957, u'IVM00065594', u'20160101', u'TAVG', 28.3, u'H', u'', u'S', u'', u'SPY', 4.746717, -6.660817, u'Ivory Coast', u'San Pedro')


In [60]:
# Creamos otra aplicación
from flask_restful import reqparse

app = Flask("Test")
api = restful.Api(app)
# Ahora añadimos recursos a la aplicación
class HelloWorld(restful.Resource):
    def get(self):
        # Argument parsing after ?id=
        parser = reqparse.RequestParser()
        parser.add_argument('code',required=True)
        args = parser.parse_args()
        code = args["code"]
        query = """
        SELECT temp,period
        FROM worldtemp 
        WHERE code='"""+code+"""'
        ORDER BY period
        """
        rows = cursor.execute(query)
        result = []
        for row in rows:
            result.append({"period": row[1],"temp":row[0]})
        
        return {"temp":result}
api.add_resource(HelloWorld, '/api/v1/temp')
# Definimos ports
app.run(port=8080)

In [84]:
# Explicación del código 
code = "PAR"
query = """
    SELECT temp,period
    FROM worldtemp 
    WHERE code='"""+code+"""'
    ORDER BY period
    """
rows = cursor.execute(query)

In [70]:
result=[]
for row in rows:
    dict = {}
    dict["temp"]=row[0]
    dict["period"]=row[1]
    result.append(dict)

In [85]:
# Alternativa
result=[]
for row in rows:
    result.append({"temp": row[0],"period": row[1]})
result[:5]

[{'period': u'20160101', 'temp': 6.7},
 {'period': u'20160102', 'temp': 9.1},
 {'period': u'20160103', 'temp': 7.3},
 {'period': u'20160104', 'temp': 7.4},
 {'period': u'20160105', 'temp': 7.4}]

## Mirar aplicación de "Chrome WebStore" llamada postman para probar webservices